In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *
from utterance.error_tools import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [3]:
191*120/3600

6.366666666666666

In [4]:
# 文章ごとに n-gram を考えてみる
def get_ngram_set(doc, N=3):
    if isinstance(doc, str):
        doc = nlp(doc)
    surfaces = [token.text for token in doc]
    ngram_set = set()
    filled = ["FOS", *surfaces, "EOS"]
    # print(filled)
    for i in range(len(filled)-N+1):
        f = "_".join(filled[i:i+N])
        ngram_set.add(f)
    return ngram_set

In [5]:
def check_repeat_rate(target:set, history:list, border=0.7):
    t_list = list(target)
    for prev_set in history:
        size = len(prev_set)
        hit = 0
        for t in t_list:
            if t in prev_set:
                hit+=1
        if hit/size >= border:
            return True
    
    return False

In [33]:
error = "Repetition"
y = []
for conv in convs:
    for ut in conv:
        if not ut.is_system():
            continue
        # 
        if ut.is_exist_error():
            if ut.is_error_included(error):
                # print(ut.errors)
                y.append(1)
            else:
                y.append(0)

In [34]:
len(y)

1386

In [7]:
idf_path = "../../corpus/wiki/idf_wiki_v2.json"
with open(idf_path, "r") as f:
    idf_dict = json.load(f)

In [8]:
def is_under_idf_border_morpheme(text, idf_border):
    for token in mecab_tokenize(text):
        key = token
        if key in idf_dict:
            if idf_dict[key] > idf_border:
                return False
        else:
            return False
    return True

In [40]:
def check_repeat_rate_ret_i(target:set, history:list, border=0.7):
    t_list = list(target)
    for i, prev_set in enumerate(history):
        size = len(prev_set)
        hit = 0
        for t in t_list:
            if t in prev_set:
                hit+=1
        if hit/size >= border:
            return i
    
    return -1

In [49]:
import Levenshtein

y_pred = []
border = 0.8

border_10 = 14.62592894814336
border_1 = 9.88253708481772
border_005 = 5.340630805533431

pair_list = []

for conv in tqdm(convs):
    ngram_sets = []
    prev_sents = []
    for ut in conv:
        if not ut.is_system():
            continue
        utt = ut.utt
        doc = nlp(utt)
        if ut.is_exist_error():
            y_pred.append(0)
        
        for sent in doc.sents:
            # idf が小さい場合は，スルーしましょう
            if is_under_idf_border_morpheme(sent.orth_, border_005):
                # print(sent)
                continue
            ngram_set = get_ngram_set(sent, N=3)
            # これまでのセットで重複が大きいものがあるかチェック！
            if ut.is_exist_error():
                ngram_checked = check_repeat_rate_ret_i(target=ngram_set, history=ngram_sets, border=0.8)
                # if check_repeat_rate_ret_i(target=ngram_set, history=ngram_sets, border=0.8):
                    # print(ut, ut.errors)
                if ngram_checked >= 0:
                    y_pred[-1] = 1
                    pair_list.append([sent.orth_, prev_sents[ngram_checked], "ngram"])

                if y_pred[-1] == 0:
                    for i, prev in enumerate( prev_sents ) :
                        if Levenshtein.ratio(sent.orth_, prev) >= 0.8:
                            pair_list.append([sent.orth_, prev_sents[i], "leven"])
                            y_pred[-1] = 1
                

            ngram_sets.append(ngram_set)
            prev_sents.append(sent.orth_)

100%|██████████| 200/200 [00:29<00:00,  6.73it/s]


In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import metrics
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[1296   34]
 [  10   46]]
accuracy =  0.9682539682539683
precision =  0.575
recall =  0.8214285714285714
f1 score =  0.676470588235294


- ngram  border 0.8

        confusion matrix = 
        [[2131   21]
        [  29   19]]
        accuracy =  0.9772727272727273
        precision =  0.475
        recall =  0.3958333333333333
        f1 score =  0.4318181818181817

    - 再現率が欲しい

- Levenshtein bordr 0.8
    
        confusion matrix = 
        [[2102   50]
        [  19   29]]
        accuracy =  0.9686363636363636
        precision =  0.3670886075949367
        recall =  0.6041666666666666
        f1 score =  0.45669291338582674

    - 悪くないが，適合率が低い

- ngram  border 0.8 and Levenshtein bordr 0.8 and idf 005%
    - 仮定 : 「そうですよね」のように，全く情報量がない発話が繰り返された場合は不要な繰り返しの可能性が低い
        - idf 辞書の活用
    - 予想 : 誤検出が減る
    
            confusion matrix = 
            [[2091   61]
            [  15   33]]
            accuracy =  0.9654545454545455
            precision =  0.35106382978723405
            recall =  0.6875
            f1 score =  0.46478873239436624


    - 評価データでの実行
            
            confusion matrix = 
            [[2080   64]
            [  10   46]]
            accuracy =  0.9663636363636363
            precision =  0.41818181818181815
            recall =  0.8214285714285714
            f1 score =  0.5542168674698795


    - 誤検出が増えたが，未検出が減った
        - len(sent) <3 の中に，情報量があり，似た発話があるが不要ではない
        - len(sent) <3 の中に検出するべきエラーが存在した
    
    - 今後
        - どのような発話が雑談において不要ではないのかの検討が出来ていない
            - 共起辞書が活用できるか？
                - データを見た限り，そう簡単にはいかない
        - 非常に近い距離で，動詞や形容詞だけが違うという場合が検出出来ていない
            - 検出された発話中で，名詞や動詞などを比較するべきか

In [50]:
i = 0
j = 0
for conv in convs:
    conv_list = []
    for ut in conv:
        conv_list.append(ut.utt)
        if not ut.is_system():
            continue
        # 本来エラーではないけどエラー扱い
        if ut.is_exist_error():
            if y[i]==0 and y_pred[i]==1:
                print("本来エラーではないけどエラー扱い", ut.did)
                print(pair_list[j])
                print()
            # if y[i]==1 and y_pred[i]==0:
            #     print("本来エラーなのに非エラー扱い", ut.did)
            #     print(conv_list)
            #     print()
            if y[i]==1 and y_pred[i]==1:
                print("よく検出した！えらいぞ", ut.did)
                print(conv_list)
                print(pair_list[j])
                print()
            
            if y_pred[i]==1:
                j += 1
            
            i+=1

よく検出した！えらいぞ 1502868266
['こんにちは。ちょうど退屈してたんだ。', 'こんにちは。元気ですか？', '元気ですかは元気です', '元気そうでよかった。', '元気ですかは元気ですね']
['元気ですかは元気ですね', '元気ですかは元気です', 'ngram']

よく検出した！えらいぞ 1502868266
['こんにちは。ちょうど退屈してたんだ。', 'こんにちは。元気ですか？', '元気ですかは元気です', '元気そうでよかった。', '元気ですかは元気ですね', 'いまなにしてたの？', 'もちろんですってのが元気ですかにいいですね', 'そうなんだ。どうして？', 'もちろんですってのが元気ですかにいいらしいですよ']
['もちろんですってのが元気ですかにいいらしいですよ', 'もちろんですってのが元気ですかにいいですね', 'leven']

本来エラーではないけどエラー扱い 1502869359
['夏休みは楽しいですね', '夏休みは欲しいですね', 'leven']

よく検出した！えらいぞ 1470623771
['こんにちは。海へ行きたいね。', '海はいいですね。泳ぎは得意ですか。', '泳ぎはいいですね', '気持ちいいですよね。', '泳ぎが上手ですね', 'ありがとう。クロールが得意なんです。', 'クロールは得意ですね', 'そう、スイミングで習ってるんです。', '朝からスイミングで泳ぎます？？', '泳ぎますよ。目が覚めていいですよ。', '目は覚めてるんですね。スイミングは楽しいですね', 'ですよね。そういえばオリンピックの水泳も楽しみですね。', 'スイミングがいいですね', '金メダルとれるといいですよね。', 'スイミングは楽しいですね']
['スイミングは楽しいですね', 'スイミングは楽しいですね', 'ngram']

よく検出した！えらいぞ 1502868964
['こんにちは。声かけてくれるのを待ってたんだ。', 'こんにちは。調子はどうですか？', '何か普段の生活で気をつけていることはありますか。', '健康には特に気を付けています。', '和食、洋食、中華どれが食べたいですか？', 'そうですねぇ。和食ですかね。', '和食ですか。和食でヘルシーですね', 'そうですね。あなたはど

In [16]:
# 共起辞書の活用
ppmi_dataname = "../../corpus/collocation/ppmi_ntt1"
ppmi_matrix = np.load(ppmi_dataname+".npy")

In [17]:
dictname = "../../corpus/collocation/word_dict/ppmi_word_dict_ntt1.json"
with open(dictname, "r") as f:
    word_dict = json.load(f)

In [18]:
def ppmi(ppmi_matrix, word_dict, x, y):
    if x not in word_dict.keys():
        return 0
    elif y not in word_dict.keys():
        return 0

    x_id = word_dict[x]
    y_id = word_dict[y]
    return ppmi_matrix[x_id, y_id]

In [19]:
ppmi(ppmi_matrix, word_dict, "祭り", "掛け声")

0

In [21]:
toyoshima_set = set("NOUN PROPN VERB ADJ".split())

def filtering(doc, filter_set):
    left = []
    for token in doc:
        if token.pos_ in filter_set:
            left.append(token.lemma_)
    return left if len(left)>0 else ["[NONE]"]

In [28]:
def colocate_rate(utt1, utt2):
    left_1 = filtering( nlp(utt1) , toyoshima_set)
    left_2 = filtering( nlp(utt2) , toyoshima_set)
    
    # 名詞がないよ
    if left_1[0] == "[NONE]" or  left_2[0] == "[NONE]":
        return 0

    # print(left_1, left_2)
    rate = 0
    for lx in left_1:
        for ly in left_2:
            rate += ppmi(ppmi_matrix, word_dict, lx, ly)
    rate = rate/(len(left_1)+len(left_2))
    return rate

In [51]:
from gensim.models import KeyedVectors
w2v_path = "../../corpus/w2v/"
# fasttext
# https://qiita.com/Hironsan/items/513b9f93752ecee9e670
# w2v_name =  "dep-ja-300dim"
w2v_name =  "model.vec"
w2v_model = KeyedVectors.load_word2vec_format(w2v_path+w2v_name)

[14630] 2022-01-13 00:01:22,978 Info gensim.models.keyedvectors :loading projection weights from ../../corpus/w2v/model.vec
[14630] 2022-01-13 00:02:36,465 Info gensim.utils :KeyedVectors lifecycle event {'msg': 'loaded (351122, 300) matrix of type float32 from ../../corpus/w2v/model.vec', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-01-13T00:02:36.465444', 'gensim': '4.0.1', 'python': '3.6.9 (default, Jan 26 2021, 15:33:00) \n[GCC 8.4.0]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-Ubuntu-18.04-bionic', 'event': 'load_word2vec_format'}


In [52]:
def w2v(word, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    if word in SYMBOL_w2v:
        vector = SYMBOL_w2v[word]
    elif word in w2v_model:
        vector = w2v_model[word]
    else:
        vector = SYMBOL_w2v["[UNK]"]
    return vector

In [53]:
wsv_dim = w2v_model["あ"].shape[0]
add_keys = ["FOS", "EOS", "[SEP]", "[UNK]", "[NONE]"]
add_weights = [np.random.randn(wsv_dim) for _ in range(len(add_keys))]
add_weights = [ v/np.linalg.norm(v) for v in add_weights ]
SYMBOL_w2v = dict(zip(add_keys, add_weights))

In [54]:
def doc2vec2(doc, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    left = filtering(doc, independent_set)
    return np.mean([ w2v(w, w2v_model, SYMBOL_w2v) for w in left], axis=0)

In [56]:
def sentence2formated(sen, w2v_model:KeyedVectors, SYMBOL_w2v:dict):
    docs = sentence2docs(sen, sents_span=False)
    vector = []
    for i, doc in enumerate(docs):
        if i==0:
            prev_vector = doc2vec2(doc, w2v_model, SYMBOL_w2v)
        else:
            current_vector = doc2vec2(doc, w2v_model, SYMBOL_w2v)
            diff_vec = np.abs(prev_vector-current_vector)       
            vector.append( diff_vec)
    return vector

In [81]:
i = 0
j = 0
y_pred2 = []
for conv in convs:
    conv_list = []
    for ut in conv:
        conv_list.append(ut.utt)
        if not ut.is_system():
            continue
        # 本来エラーではないけどエラー扱い
        if ut.is_exist_error():
            if y_pred[i]==1:
                # ペアをベクトル化し，差分を計算
                # print(pair_list[j][:2])
                vector = sentence2formated(pair_list[j][:2], w2v_model, SYMBOL_w2v)[0]
                # print(np.linalg.norm(vector))
                if np.linalg.norm(vector) > 1.5:
                    y_pred2.append(0)
                    if y[i] == 1:
                        print( pair_list[j][:2] ) 
                else:
                    y_pred2.append(1)
                j += 1
            else:
                y_pred2.append(0)
            
            i+=1

['自動販売機は何気ないですね', '自動販売機はいいですね']
['イタリア料理は得意ですね', 'イタリア料理は好きですね']
['ネタが美味しいですね', '尻尾が美味しいですね']
['手洗いうがいで予防が大切ですね', '予防が大切ですね']
['寝不足で頭が痛いですか', '寝不足で頭が重いですか']
['紅茶飲料の「冷え知らず」', '「冷え知らず」']


In [73]:
score(y, y_pred2)

confusion matrix = 
 [[1309   21]
 [  16   40]]
accuracy =  0.9733044733044733
precision =  0.6557377049180327
recall =  0.7142857142857143
f1 score =  0.6837606837606838


In [76]:
doc = nlp("かけ声を覚えたことあります")
filtering(doc, independent_set)

['かけ声', '覚える', 'こと', 'ある']